# Metodos no lineales para reduccion de la dimensionalidad

En esta experimentacion nos proponemos experimentar con metodos alternativos de preprocesamiento de los datos para reducir su dimension. Consideraremos los siguientes metodos (EXPLICARLOS EN EL INFORME)

-tsne sklearn.manifold: Manifold Learning? (explorar)
-lda
-autoencoders


Una de las hipotesis que obviamente tengo es que las relaciones entre pixeles y la clase no necesariamente es lineal (y lo mas probable es que no lo sea)

A PARTIR DE ACA NUESTRA REFERENCIA 'OPTIMA' COMO ALGORITMO DE PCA Y KNN VA A SER EL DE ALFA 15 Y K 15

A partir de ahora solo compararemos accuracy y tiempo de ejecucion (recordemos que el resto de las metricas eran practicamente iguales para este dataset).

- Classify images with TPUs in [**Petals to the Metal**](https://www.kaggle.com/c/tpu-getting-started)
- Create art with GANs in [**I'm Something of a Painter Myself**](https://www.kaggle.com/c/gan-getting-started)
- Classify Tweets in [**Real or Not? NLP with Disaster Tweets**](https://www.kaggle.com/c/nlp-getting-started)
- Detect contradiction and entailment in [**Contradictory, My Dear Watson**](https://www.kaggle.com/c/contradictory-my-dear-watson)